# Controllable loop process
> iteration with callbacks

In [1]:
# default_exp loop

In [1]:
# export 
import numpy as np

In [46]:
# export
from tqdm import tqdm
from types import MethodType
import inspect
from functools import partial

def method4all(f):
    """
    Use this function as a decorator,
    The decorated function under Loop class can be used on outer layer
    """
    setattr(f,"forall",True)
    return f

class StorageCore:
    def __init__(self,start_layer):
        self.layers = []
        self.lmap = dict()
        self.forall_pool = dict()
        self.new_layer(start_layer)
        self.i = -1
        self.epoch = -1
    
    def new_layer(self,layer):
        layer.core = self
        self.layers.append(layer)
        self.lmap[layer._loop_layer]=layer
        if hasattr(self,layer.name):
            raise KeyError(f"Layer name already exist: {layer.name}")
        setattr(self,layer.name,layer)
        self.update_forall(layer)
        self.assign_forall(layer)
        
    def __repr__(self):
        return str(self.lmap)
    
    def immitate_func(self,obj,name,func):
        setattr(obj,name,func)
        
    def for_all_functions(self,obj):
        return dict(inspect.getmembers(obj, 
                                  predicate=lambda x:hasattr(x,"forall") if inspect.ismethod(x) else False))
    
    def update_forall(self,obj):
        self.forall_pool.update(self.for_all_functions(obj))
        
    def assign_forall(self,obj):
        for name,f in self.forall_pool.items():
            self.immitate_func(obj,name,f)

class Loop:
    """
    Basic loop class
    """
    _loop_layer = 0.
    def __init__(self,iterable = [],name=None):
        self.iterable  = iterable
        self.name = name if name!=None else self.__class__.__name__
        
        if hasattr(iterable,"_loop_layer"):
            self._loop_layer = iterable._loop_layer + 1
            iterable.core.new_layer(self)
        else:
            self.core = StorageCore(self)
        
    def __call__(self):
        pass
    
    def __repr__(self):
        return f"layer:>>>{self.name}"
    
    def on(self,iterable):
        self.iterable = iterable
        self._loop_layer = iterable._loop_layer + 1
        
    def func_detail(self,f):
        detail = dict({"🐍func_name":f.__name__,
                       "⛰doc":f.__doc__,
                       "😝var":",".join(f.__code__.co_varnames),
                       "😜names":",".join(f.__code__.co_names),
                      })
        return detail
    
    def summary(self):
        rt = f"Loop layer {self.name} summary:\n"
        rt+= "="*50+"\n"
        funcs = []
        for idx,layer in self.core.lmap.items():
            rt+= f"🍰layer{idx}\t{str(layer)}\n"
            for fname,f in self.core.for_all_functions(layer).items():
                if id(f) not in funcs:
                    rt+="\t"
                    rt+="\n\t".join(f"[{k}]\t{v}" for k,v in self.func_detail(f).items())
                    rt+="\n\t"+"."*35+"\n"
                    funcs.append(id(f))
            rt+="-"*50+"\n"
        rt+= "="*50+"\n"
        print(rt)
    
    def __len__(self):
        return self.iterable.__len__()
    
    def run(self,):
        for element in self:
            pass
        
    def update_i(self):
        if self.is_newloop():
            self.core.i=-1
            self.core.epoch+=1
        self.core.i+=1
            
    def __iter__(self,):
        for element in self.iterable:
            if self._loop_layer ==0:
                self.update_i()
            self.core.element = element
            self()
            yield self.element
            
    def __getattr__(self,k):
        return getattr(self.core,k)
    
    def is_newloop(self):
        """
        return Bool:Is this a new loop ready to start
        """
        return (self.i==-1 or self.i==self.__len__()-1)

## Loop-in-loop concept
Loop and its inheriting descendants runs on iterable, incuding other ```Loop```

In [47]:
l = Loop(range(5,10),name = "hahahha")
l

layer:>>>hahahha

In [48]:
class Printer(Loop):
    def __call__(self):
        print(f"epoch_{self.core.epoch}",end="\t")
        print(f"i_{self.core.i}",end="\t")
        print(self.core.element)
        
l2 = Printer(l)
l2.run()

epoch_0	i_0	5
epoch_0	i_1	6
epoch_0	i_2	7
epoch_0	i_3	8
epoch_0	i_4	9


## Progress bar

In [49]:
# export
class ProgressBar(Loop):
    def __init__(self,iterable=[],jupyter = True):
        super().__init__(iterable,"Progressb Bar")
        
        if jupyter: # jupyter widget
            from tqdm.notebook import tqdm as tq
            
        else: # compatible for console print
            from tqdm import tqdm as tq
            
        self.tq = tq
        self.data = dict()
        
    @method4all
    def pgbar_data(self,data):
        """
        update progress bar with python dictionary
        data: python dictionary
        """
        self.t.set_postfix(data)
    
    @method4all
    def pgbar_description(self,text):
        """
        update progress bar prefix with text string
        """
        self.t.set_description_str(f"{text}")
        
    def __iter__(self):
        # initialize 
        if self.is_newloop():
            self.create_bar()
            
        # loop through
        for element in self.t:
            self()
            yield element
            
    def create_bar(self):
        self.t = self.tq(self.iterable)
        self.gen = iter(self.t)
            


In [50]:
ProgressBar(Loop(range(5,10)),jupyter = True).run()

### Application upon progress bar
The function ```pgbar_data``` is decorated under ```method4all```, hence the function can be used else where

In [51]:
from datetime import datetime
class Application(Loop):
    def __call__(self):
        # create some data
        loop_data = dict(epoch=self.epoch,i = self.i,
                         val = self.element,)
        # update data to progress bar
        self.pgbar_data(loop_data)
        self.pgbar_description(str(datetime.now().strftime("%H:%M:%S")))

l3 = Application(ProgressBar(Loop(range(5,10)),jupyter=True))
l3.run()

## Print out the summary of the loop

In [52]:
l3.summary()

Loop layer Application summary:
🍰layer0.0	layer:>>>Loop
--------------------------------------------------
🍰layer1.0	layer:>>>Progressb Bar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer:>>>Application
--------------------------------------------------



In [53]:
l3.run()

## Event

Event allows add-hoc callbacks to be created

In [54]:
# export
class Event(Loop):
    """
    An event is the landmark with in 1 iteration
    """
    def __init__(self,iterable=[],event_name = None,cbs = []):
        super().__init__(iterable,event_name)
        self.cbs = cbs
        
        def call(self):
            return self.__call__()
        call.__name__ = f"on_{event_name}"
        call.__doc__ = f"""
            Excute callback for event:{event_name}
        """
        setattr(self,call.__name__,MethodType(method4all(call),self))
        
        def set_(self,f):
            return self.on(f)
        set_.__name__ = f"set_{event_name}"
        set_.__doc__ = f"""
            Append new callback for event:{event_name}
            Use this function as decorator
        """
        setattr(self,set_.__name__,MethodType(method4all(set_),self))
    
    def __call__(self):
        for cb in self.cbs:
            cb(self)
       
    def on(self,f):
        def wrapper(self):
            return f(self)
        self.cbs.append(wrapper)
        return wrapper

In [55]:
class ToString(Loop):
    def __call__(self,):
        self.core.element = f"variable_{self.i}"

class AfterToString(Event):
    def __init__(self,iterable=[],cbs=[]):
        super().__init__(iterable=iterable,event_name="after_to_string",cbs=cbs)

def get_frame(iterable):
    l1 = ProgressBar(Loop(iterable,"base"))
    l2 = ToString(l1,)
    l3 = AfterToString(l2)
    return l3

In [56]:
l = get_frame(range(10))

In [57]:
# you can use this "on" syntax
@l.after_to_string.on
def trim(self):
    self.core.element = f"Var_{self.element}"
    
# or this syntax
@l.set_after_to_string
def update(self,):
    self.pgbar_description(self.element)

In [58]:
l.summary()

Loop layer after_to_string summary:
🍰layer0.0	layer:>>>base
--------------------------------------------------
🍰layer1.0	layer:>>>Progressb Bar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer:>>>ToString
--------------------------------------------------
🍰layer3.0	layer:>>>after_to_string
	[🐍func_name]	on_after_to_string
	[⛰doc]	
            Excute callback for event:after_to_string
        
	[😝var]	self
	[😜names]	__call__
	...................................
	[🐍func_name]	set_after_to_string
	[⛰doc]	
            Append new callback for event:after_to_string
            Use this f

In [59]:
l.run()